<a href="https://colab.research.google.com/github/wint3rx3/quiz-whale-ai/blob/main/yohan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [3]:
!git clone https://github.com/wint3rx3/quiz-whale-ai

fatal: destination path 'quiz-whale-ai' already exists and is not an empty directory.


In [10]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import json
import re

def process_equations(text):
    """
    추출된 텍스트 내에서 수식으로 판단되는 부분을 Markdown 수식 형식으로 감쌉니다.
    예시에서는 \frac{...} 패턴이나 ∑, ∫ 기호가 포함된 경우 처리합니다.
    이미 수식 형식($ 또는 $$)으로 감싸져 있다면 건드리지 않습니다.
    """
    # \frac 패턴이 있으면 블록 수식으로 처리
    if re.search(r'\\frac\{', text):
        if not (text.startswith("$$") and text.endswith("$$")):
            text = "$$" + text + "$$"
    # 수학 기호가 포함된 경우 인라인 수식 처리
    elif re.search(r'[∑∫]', text):
        if not (text.startswith("$") and text.endswith("$")):
            text = "$" + text + "$"
    return text

def clean_text(text):
    """
    1. 제어 문자(ASCII 0~31) 제거 및 불필요한 공백 정리
    2. 수식으로 보이는 부분을 process_equations()를 통해 Markdown 형식으로 감쌈
    """
    cleaned = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]+', '', text)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    cleaned = process_equations(cleaned)
    return cleaned

def pdf_to_json_range(pdf_path, json_path, start_page, end_page):
    """
    PDF 파일(pdf_path)에서 start_page부터 end_page까지(포함) 텍스트를 추출하여,
    각 페이지를 { "page_number": 실제 페이지 번호, "content": 정제된 텍스트 } 형태의 JSON 파일(json_path)로 저장합니다.
    start_page와 end_page는 실제 PDF 상의 페이지 번호(1부터 시작)입니다.
    """
    data = []
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        total_pages = len(reader.pages)
        if start_page < 1 or end_page > total_pages or start_page > end_page:
            raise ValueError(f"유효한 페이지 범위를 입력하세요 (1 ~ {total_pages})")
        
        for page_number in range(start_page, end_page + 1):
            raw_text = reader.pages[page_number - 1].extract_text() or ""
            text = clean_text(raw_text)
            data.append({
                "page_number": page_number,
                "content": text
            })
    
    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

# 예시 실행: sample.pdf 파일의 3페이지부터 6페이지까지 추출
pdf_path = '/content/drive/MyDrive/Colab Notebooks/quiz-whale-ai/transformer.pdf'
json_path = 'sample.json'
start_page = 1
end_page = 11
pdf_to_json_range(pdf_path, json_path, start_page, end_page)